In [5]:
import os
import csv
import json
import time
import numpy as np

from wrapper import read, pretty_file_name, solver, mapf, base_ipf, pe_simple_threshold, global_heatmap, partial_solving, select_portion_based_on_heatmap
from os import path
from multiprocessing import Pool

In [6]:
instances_pool_path = 'instances'
instances_pool = [(pretty_file_name(file), read(path.join(instances_pool_path, file))) for file in os.listdir(
    instances_pool_path) if path.isfile(path.join(instances_pool_path, file))]

np.random.shuffle(instances_pool)

timeout = 60 * 15
header = ['Step Name', 'Total Time', 'Ground Time', 'Solving Time', 'SAT', 'Additional Data']

# row = [{
#     'Step Name' : ,
#     'Total Time' :,
#     'Ground Time': ,
#     'Solving Time': ,
#     'SAT' : ,
#     'Additional Data' :
# }]


In [7]:
def write_csv(filename:str,header:list[str],rows):
    with open(file=filename, mode='w') as benchmark_csv:
        benchmark_writer = csv.DictWriter(benchmark_csv, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL, fieldnames=header)
        benchmark_writer.writeheader()
        for row in rows:
            benchmark_writer.writerow(row)
            


In [8]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        rows = []
        print(instance_name)
        r, s = mapf(instance_content)
    
        rows.append({
            'Step Name' : 'base_mapf',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        write_csv(f'benchmarks_result/{instance_name}-base_mapf.csv', header,rows)
        
        for i in range (1,6,2):
            
                r_,s = mapf(instance_content, horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'mapf_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })

        write_csv(f'benchmarks_result/{instance_name}-base_mapf.csv', header,rows)

    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()
    

random_s40_a35_c75_h54
Grounding...


Solving...
Grounding...
Solving...
Grounding...

random_s20_a15_c50_h43
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
random_x20_y20_a15_c85_h21
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
room_x20_y20_a30_w5_h36
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
room_x15_y15_a30_w5_h22
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
random_x15_y15_a10_c85_h19
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
room_x10_y15_a40_w5_h23
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
random_s10_a5_c50_h16
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
room_s47_a5_w5_h50
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solving...
Grounding...
Solv

In [9]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content,npaths=1)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
  
        r,s = solver(r+instance_content, 'pathselection/random_paths_selection.lp', parameters=['0','--opt-strategy=usc', f'--seed={int(time.time())}', "--init-watches=rnd"])
        
        rows.append({
            'Step Name' : 'random_path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })


        r_,s = partial_solving(r , 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp',precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):

            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        

        write_csv(f'benchmarks_result/{instance_name}-witness.csv', header,rows)

        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54
Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:03<00:00,  9.79it/s]


Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_s20_a15_c50_h43
43
Grounding...


Current agent: 15 - at((18,7),0) => at((17,17),43): 100%|██████████| 15/15 [00:00<00:00, 53.82it/s] 


Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_x20_y20_a15_c85_h21
21
Grounding...


...

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_x25_y25_a20_c75_h36
36
Grounding...


Current agent: 20 - at((1,9),0) => at((3,10),36): 100%|██████████| 20/20 [00:00<00:00, 28.90it/s]   


Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 19.93it/s]


Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 1 : No additional path, heatmap simple threshold elimination, pre-computed path on full graph

In [10]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        # print(r)
        
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph=False, precomputed_path='mapf/precomputed_path_allgraph.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph=False, precomputed_path='mapf/precomputed_path_allgraph.lp',  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        

        write_csv(f'benchmarks_result/{instance_name}-approach01.csv', header,rows)

        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54


Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:04<00:00,  7.82it/s]


SAT
2064 38 0.02857142857142857
[((36, 17), 53), ((36, 17), 52), ((36, 17), 51), ((36, 17), 50), ((36, 17), 49), ((36, 17), 48), ((36, 17), 47), ((36, 17), 46), ((36, 17), 45), ((36, 17), 44), ((36, 17), 43), ((36, 17), 42), ((36, 17), 41), ((36, 17), 40), ((15, 10), 40), ((36, 17), 39), ((22, 24), 39), ((36, 17), 38), ((36, 17), 37), ((36, 17), 36), ((36, 17), 35), ((36, 17), 34), ((36, 17), 33), ((36, 17), 32), ((36, 17), 31), ((36, 17), 30), ((36, 17), 29), ((24, 16), 29), ((36, 20), 26), ((36, 17), 23), ((37, 12), 17), ((28, 24), 3), ((34, 7), 10), ((26, 19), 6), ((37, 12), 35), ((26, 6), 41), ((36, 16), 5), ((35, 10), 8)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...

random_s20_a15_c50_h43
43
Grounding...


Current agent: 15 - at((18,7),0) => at((17,17),43): 100%|██████████| 15/15 [00:00<00:00, 40.04it/s]


...

SAT
103 8 0.2
[((8, 10), 5), ((8, 10), 4), ((8, 10), 3), ((8, 10), 6), ((6, 3), 8), ((8, 10), 7), ((9, 10), 2), ((9, 9), 1)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_x25_y25_a20_c75_h36
36
Grounding...


Current agent: 20 - at((1,9),0) => at((3,10),36): 100%|██████████| 20/20 [00:00<00:00, 21.05it/s]   


SAT
999 7 0.05
[((21, 7), 26), ((6, 15), 12), ((11, 5), 4), ((4, 10), 17), ((11, 4), 17), ((11, 5), 16), ((5, 10), 18)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 11.69it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 2: No additional path, heatmap simple threshold elimination, subgraph approach

In [11]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        # r,s = solver(r,'mapf/add_lowest_heatmap_path.lp')
        
        print()
        # print(r)
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        print('\n')
        write_csv(f'benchmarks_result/{instance_name}-approach02.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54
Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:04<00:00,  8.23it/s]


SAT
2064 38 0.02857142857142857
[((36, 17), 53), ((36, 17), 52), ((36, 17), 51), ((36, 17), 50), ((36, 17), 49), ((36, 17), 48), ((36, 17), 47), ((36, 17), 46), ((36, 17), 45), ((36, 17), 44), ((36, 17), 43), ((36, 17), 42), ((36, 17), 41), ((36, 17), 40), ((15, 10), 40), ((36, 17), 39), ((22, 24), 39), ((36, 17), 38), ((36, 17), 37), ((36, 17), 36), ((36, 17), 35), ((36, 17), 34), ((36, 17), 33), ((36, 17), 32), ((36, 17), 31), ((36, 17), 30), ((36, 17), 29), ((24, 16), 29), ((36, 20), 26), ((36, 17), 23), ((37, 12), 17), ((28, 24), 3), ((34, 7), 10), ((26, 19), 6), ((37, 12), 35), ((26, 6), 41), ((36, 16), 5), ((35, 10), 8)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_s20_a15_c50_h43
43
Grounding...


Current agent: 15 - at((18,7),0) => at((17,17),43): 100%|██████████| 15/15 [00:00<00:00, 32.07it/s] 


SAT
739 34 0.06666666666666667
[((16, 15), 20), ((19, 7), 41), ((2, 8), 37), ((18, 11), 36), ((2, 7), 36), ((2, 6), 35), ((2, 5), 34), ((2, 4), 33), ((2, 3), 32), ((2, 2), 31), ((3, 2), 30), ((4, 2), 29), ((14, 11), 27), ((14, 11), 26), ((18, 11), 23), ((19, 7), 18), ((14, 11), 14), ((17, 18), 14), ((19, 7), 8), ((19, 15), 2), ((11, 11), 1), ((17, 18), 42), ((19, 7), 24), ((18, 11), 29), ((11, 11), 29), ((19, 15), 7), ((14, 11), 32), ((18, 11), 30), ((12, 11), 29), ((14, 11), 31), ((17, 18), 41), ((19, 14), 8), ((16, 15), 35), ((19, 7), 25)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_x20_y20_a15_c85_h21
21
Grounding...


...

SAT
103 8 0.2
[((8, 10), 5), ((8, 10), 4), ((8, 10), 3), ((8, 10), 6), ((6, 3), 8), ((8, 10), 7), ((9, 10), 2), ((9, 9), 1)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_x25_y25_a20_c75_h36
36
Grounding...


Current agent: 20 - at((1,9),0) => at((3,10),36): 100%|██████████| 20/20 [00:01<00:00, 19.47it/s]   


SAT
999 7 0.05
[((21, 7), 26), ((6, 15), 12), ((11, 5), 4), ((4, 10), 17), ((11, 4), 17), ((11, 5), 16), ((5, 10), 18)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 14.71it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True




## Approach 3: no additional paths, simple threshold elimination, precomputed path on subgraph

In [12]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,           
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        print()
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp', horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        write_csv(f'benchmarks_result/{instance_name}-approach03.csv', header,rows)
        
    
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54


Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:04<00:00,  8.38it/s]


SAT
2064 38 0.02857142857142857
[((36, 17), 53), ((36, 17), 52), ((36, 17), 51), ((36, 17), 50), ((36, 17), 49), ((36, 17), 48), ((36, 17), 47), ((36, 17), 46), ((36, 17), 45), ((36, 17), 44), ((36, 17), 43), ((36, 17), 42), ((36, 17), 41), ((36, 17), 40), ((15, 10), 40), ((36, 17), 39), ((22, 24), 39), ((36, 17), 38), ((36, 17), 37), ((36, 17), 36), ((36, 17), 35), ((36, 17), 34), ((36, 17), 33), ((36, 17), 32), ((36, 17), 31), ((36, 17), 30), ((36, 17), 29), ((24, 16), 29), ((36, 20), 26), ((36, 17), 23), ((37, 12), 17), ((28, 24), 3), ((34, 7), 10), ((26, 19), 6), ((37, 12), 35), ((26, 6), 41), ((36, 16), 5), ((35, 10), 8)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_s20_a15_c50_h43
43
Grounding...


Current agent: 15 - at((18,7),0) => at((17,17),43): 100%|██████████| 15/15 [00:00<00:00, 41.18it/s]


...

SAT
103 8 0.2
[((8, 10), 5), ((8, 10), 4), ((8, 10), 3), ((8, 10), 6), ((6, 3), 8), ((8, 10), 7), ((9, 10), 2), ((9, 9), 1)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_x25_y25_a20_c75_h36
36
Grounding...


Current agent: 20 - at((1,9),0) => at((3,10),36): 100%|██████████| 20/20 [00:00<00:00, 20.31it/s]   


SAT
999 7 0.05
[((21, 7), 26), ((6, 15), 12), ((11, 5), 4), ((4, 10), 17), ((11, 4), 17), ((11, 5), 16), ((5, 10), 18)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 16.00it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 4 : additional path, heatmap simple threshold elimination, pre-computed path on full graph

In [13]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content, additional_paths_computation=True)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        

        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph=False, precomputed_path='mapf/precomputed_path_allgraph.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph=False, precomputed_path='mapf/precomputed_path_allgraph.lp', horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
                
        print('\n\n\n')
        write_csv(f'benchmarks_result/{instance_name}-approach04.csv', header,rows)
        
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])
    # mutex.get(timeout=timeout)
    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print('Error',e)
        pool.terminate()

random_s40_a35_c75_h54
54


Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:08<00:00,  4.17it/s]


SAT
2795 50 0.02857142857142857
[((36, 17), 53), ((36, 17), 52), ((36, 17), 51), ((36, 17), 50), ((36, 17), 49), ((36, 17), 48), ((36, 17), 47), ((36, 17), 46), ((36, 17), 45), ((36, 17), 44), ((36, 17), 43), ((36, 17), 42), ((36, 17), 41), ((36, 17), 40), ((36, 17), 39), ((22, 24), 39), ((36, 17), 38), ((36, 17), 37), ((36, 17), 36), ((36, 17), 35), ((36, 17), 34), ((36, 17), 33), ((36, 17), 32), ((36, 17), 31), ((36, 17), 30), ((36, 17), 29), ((28, 24), 5), ((34, 7), 10), ((15, 10), 42), ((15, 10), 40), ((23, 16), 30), ((24, 16), 29), ((36, 19), 27), ((36, 20), 26), ((36, 17), 25), ((36, 17), 23), ((37, 12), 19), ((37, 12), 17), ((35, 10), 2), ((34, 8), 11), ((34, 7), 12), ((37, 12), 37), ((37, 12), 35), ((26, 6), 43), ((26, 6), 41), ((32, 4), 2), ((28, 24), 4), ((28, 24), 3), ((33, 4), 3), ((34, 7), 11)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Error 
random_s20_a15_c50

Current agent: 15 - at((18,7),0) => at((17,17),43): 100%|██████████| 15/15 [00:00<00:00, 29.45it/s]


...

SAT
103 8 0.2
[((8, 10), 5), ((8, 10), 4), ((8, 10), 3), ((8, 10), 6), ((6, 3), 8), ((8, 10), 7), ((9, 10), 2), ((9, 9), 1)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True




random_x25_y25_a20_c75_h36
36
Grounding...


Current agent: 20 - at((1,9),0) => at((3,10),36): 100%|██████████| 20/20 [00:01<00:00, 16.31it/s]   


SAT
1170 13 0.05
[((21, 7), 26), ((4, 10), 17), ((11, 5), 6), ((7, 15), 13), ((6, 15), 12), ((13, 21), 4), ((13, 22), 3), ((4, 14), 2), ((4, 14), 1), ((11, 5), 16), ((11, 5), 4), ((11, 4), 17), ((5, 10), 18)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True




room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.35it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True






## Approach 5:  additional path, heatmap simple threshold elimination, subgraph approach

In [14]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content, additional_paths_computation=True)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        

        write_csv(f'benchmarks_result/{instance_name}-approach05.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54


Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:08<00:00,  4.19it/s]


SAT
2795 50 0.02857142857142857
[((36, 17), 53), ((36, 17), 52), ((36, 17), 51), ((36, 17), 50), ((36, 17), 49), ((36, 17), 48), ((36, 17), 47), ((36, 17), 46), ((36, 17), 45), ((36, 17), 44), ((36, 17), 43), ((36, 17), 42), ((36, 17), 41), ((36, 17), 40), ((36, 17), 39), ((22, 24), 39), ((36, 17), 38), ((36, 17), 37), ((36, 17), 36), ((36, 17), 35), ((36, 17), 34), ((36, 17), 33), ((36, 17), 32), ((36, 17), 31), ((36, 17), 30), ((36, 17), 29), ((28, 24), 5), ((34, 7), 10), ((15, 10), 42), ((15, 10), 40), ((23, 16), 30), ((24, 16), 29), ((36, 19), 27), ((36, 20), 26), ((36, 17), 25), ((36, 17), 23), ((37, 12), 19), ((37, 12), 17), ((35, 10), 2), ((34, 8), 11), ((34, 7), 12), ((37, 12), 37), ((37, 12), 35), ((26, 6), 43), ((26, 6), 41), ((32, 4), 2), ((28, 24), 4), ((28, 24), 3), ((33, 4), 3), ((34, 7), 11)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Groun

Current agent: 15 - at((18,7),0) => at((17,17),43): 100%|██████████| 15/15 [00:00<00:00, 24.23it/s] 


...

SAT
103 8 0.2
[((8, 10), 5), ((8, 10), 4), ((8, 10), 3), ((8, 10), 6), ((6, 3), 8), ((8, 10), 7), ((9, 10), 2), ((9, 9), 1)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_x25_y25_a20_c75_h36
36
Grounding...


Current agent: 20 - at((1,9),0) => at((3,10),36): 100%|██████████| 20/20 [00:01<00:00, 13.33it/s]   


SAT
1170 13 0.05
[((21, 7), 26), ((4, 10), 17), ((11, 5), 6), ((7, 15), 13), ((6, 15), 12), ((13, 21), 4), ((13, 22), 3), ((4, 14), 2), ((4, 14), 1), ((11, 5), 16), ((11, 5), 4), ((11, 4), 17), ((5, 10), 18)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 13.72it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 6: no additional paths, simple threshold elimination, precomputed path on subgraph

In [15]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content, additional_paths_computation=True)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp', horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        write_csv(f'benchmarks_result/{instance_name}-approach06.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54


Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:08<00:00,  4.11it/s]


SAT
2795 50 0.02857142857142857
[((36, 17), 53), ((36, 17), 52), ((36, 17), 51), ((36, 17), 50), ((36, 17), 49), ((36, 17), 48), ((36, 17), 47), ((36, 17), 46), ((36, 17), 45), ((36, 17), 44), ((36, 17), 43), ((36, 17), 42), ((36, 17), 41), ((36, 17), 40), ((36, 17), 39), ((22, 24), 39), ((36, 17), 38), ((36, 17), 37), ((36, 17), 36), ((36, 17), 35), ((36, 17), 34), ((36, 17), 33), ((36, 17), 32), ((36, 17), 31), ((36, 17), 30), ((36, 17), 29), ((28, 24), 5), ((34, 7), 10), ((15, 10), 42), ((15, 10), 40), ((23, 16), 30), ((24, 16), 29), ((36, 19), 27), ((36, 20), 26), ((36, 17), 25), ((36, 17), 23), ((37, 12), 19), ((37, 12), 17), ((35, 10), 2), ((34, 8), 11), ((34, 7), 12), ((37, 12), 37), ((37, 12), 35), ((26, 6), 43), ((26, 6), 41), ((32, 4), 2), ((28, 24), 4), ((28, 24), 3), ((33, 4), 3), ((34, 7), 11)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Groun

Current agent: 15 - at((18,7),0) => at((17,17),43): 100%|██████████| 15/15 [00:00<00:00, 24.42it/s] 


...

SAT
103 8 0.2
[((8, 10), 5), ((8, 10), 4), ((8, 10), 3), ((8, 10), 6), ((6, 3), 8), ((8, 10), 7), ((9, 10), 2), ((9, 9), 1)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_x25_y25_a20_c75_h36
36
Grounding...


Current agent: 20 - at((1,9),0) => at((3,10),36): 100%|██████████| 20/20 [00:01<00:00, 15.87it/s]   


SAT
1170 13 0.05
[((21, 7), 26), ((4, 10), 17), ((11, 5), 6), ((7, 15), 13), ((6, 15), 12), ((13, 21), 4), ((13, 22), 3), ((4, 14), 2), ((4, 14), 1), ((11, 5), 16), ((11, 5), 4), ((11, 4), 17), ((5, 10), 18)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 14.43it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 7: no additional paths, simple threshold elimination, subgraph with corridor

In [16]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        r,s = solver(r,'strategies/corridor.lp', additional_encodings=['strategies/determination.lp'])
        
        rows.append({
            'Step Name' : 'strategy',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        print('\n')
        write_csv(f'benchmarks_result/{instance_name}-approach07.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54


Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:04<00:00,  8.03it/s]


SAT
2064 38 0.02857142857142857
[((36, 17), 53), ((36, 17), 52), ((36, 17), 51), ((36, 17), 50), ((36, 17), 49), ((36, 17), 48), ((36, 17), 47), ((36, 17), 46), ((36, 17), 45), ((36, 17), 44), ((36, 17), 43), ((36, 17), 42), ((36, 17), 41), ((36, 17), 40), ((15, 10), 40), ((36, 17), 39), ((22, 24), 39), ((36, 17), 38), ((36, 17), 37), ((36, 17), 36), ((36, 17), 35), ((36, 17), 34), ((36, 17), 33), ((36, 17), 32), ((36, 17), 31), ((36, 17), 30), ((36, 17), 29), ((24, 16), 29), ((36, 20), 26), ((36, 17), 23), ((37, 12), 17), ((28, 24), 3), ((34, 7), 10), ((26, 19), 6), ((37, 12), 35), ((26, 6), 41), ((36, 16), 5), ((35, 10), 8)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_s20_a15_c50_h43
43
Grounding...


Current agent: 15 - at((18,7),0) => at((17,17),43): 100%|██████████| 15/15 [00:00<00:00, 37.79it/s]


...

SAT
103 8 0.2
[((8, 10), 5), ((8, 10), 4), ((8, 10), 3), ((8, 10), 6), ((6, 3), 8), ((8, 10), 7), ((9, 10), 2), ((9, 9), 1)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_x25_y25_a20_c75_h36
36
Grounding...


Current agent: 20 - at((1,9),0) => at((3,10),36): 100%|██████████| 20/20 [00:00<00:00, 21.06it/s]   


SAT
999 7 0.05
[((21, 7), 26), ((6, 15), 12), ((11, 5), 4), ((4, 10), 17), ((11, 4), 17), ((11, 5), 16), ((5, 10), 18)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 14.85it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True




## Approach 8: no additional paths, simple threshold elimination, subgraph with diamond

In [17]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        r,s = solver(r,'strategies/diamond.lp', additional_encodings=['strategies/determination.lp'])
        
        rows.append({
            'Step Name' : 'strategy',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        print('\n')
        write_csv(f'benchmarks_result/{instance_name}-approach08.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54


Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:04<00:00,  8.37it/s]


SAT
2064 38 0.02857142857142857
[((36, 17), 53), ((36, 17), 52), ((36, 17), 51), ((36, 17), 50), ((36, 17), 49), ((36, 17), 48), ((36, 17), 47), ((36, 17), 46), ((36, 17), 45), ((36, 17), 44), ((36, 17), 43), ((36, 17), 42), ((36, 17), 41), ((36, 17), 40), ((15, 10), 40), ((36, 17), 39), ((22, 24), 39), ((36, 17), 38), ((36, 17), 37), ((36, 17), 36), ((36, 17), 35), ((36, 17), 34), ((36, 17), 33), ((36, 17), 32), ((36, 17), 31), ((36, 17), 30), ((36, 17), 29), ((24, 16), 29), ((36, 20), 26), ((36, 17), 23), ((37, 12), 17), ((28, 24), 3), ((34, 7), 10), ((26, 19), 6), ((37, 12), 35), ((26, 6), 41), ((36, 16), 5), ((35, 10), 8)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_s20_a15_c50_h43
43
Grounding...


Current agent: 15 - at((18,7),0) => at((17,17),43): 100%|██████████| 15/15 [00:00<00:00, 38.88it/s]


...

SAT
103 8 0.2
[((8, 10), 5), ((8, 10), 4), ((8, 10), 3), ((8, 10), 6), ((6, 3), 8), ((8, 10), 7), ((9, 10), 2), ((9, 9), 1)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_x25_y25_a20_c75_h36
36
Grounding...


Current agent: 20 - at((1,9),0) => at((3,10),36): 100%|██████████| 20/20 [00:01<00:00, 19.35it/s]   


SAT
999 7 0.05
[((21, 7), 26), ((6, 15), 12), ((11, 5), 4), ((4, 10), 17), ((11, 4), 17), ((11, 5), 16), ((5, 10), 18)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 14.43it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True




## Approach 9: no additional paths, simple threshold elimination, subgraph with corridor and diamond

In [18]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        r,s = solver(r,'strategies/diamond.lp', additional_encodings=['strategies/corridor.lp', 'strategies/determination.lp'])
        
        rows.append({
            'Step Name' : 'strategy',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        print('\n')
        write_csv(f'benchmarks_result/{instance_name}-approach09.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54


Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:04<00:00,  8.31it/s]


SAT
2064 38 0.02857142857142857
[((36, 17), 53), ((36, 17), 52), ((36, 17), 51), ((36, 17), 50), ((36, 17), 49), ((36, 17), 48), ((36, 17), 47), ((36, 17), 46), ((36, 17), 45), ((36, 17), 44), ((36, 17), 43), ((36, 17), 42), ((36, 17), 41), ((36, 17), 40), ((15, 10), 40), ((36, 17), 39), ((22, 24), 39), ((36, 17), 38), ((36, 17), 37), ((36, 17), 36), ((36, 17), 35), ((36, 17), 34), ((36, 17), 33), ((36, 17), 32), ((36, 17), 31), ((36, 17), 30), ((36, 17), 29), ((24, 16), 29), ((36, 20), 26), ((36, 17), 23), ((37, 12), 17), ((28, 24), 3), ((34, 7), 10), ((26, 19), 6), ((37, 12), 35), ((26, 6), 41), ((36, 16), 5), ((35, 10), 8)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_s20_a15_c50_h43
43
Grounding...


Current agent: 15 - at((18,7),0) => at((17,17),43): 100%|██████████| 15/15 [00:00<00:00, 38.18it/s]


...

SAT
103 8 0.2
[((8, 10), 5), ((8, 10), 4), ((8, 10), 3), ((8, 10), 6), ((6, 3), 8), ((8, 10), 7), ((9, 10), 2), ((9, 9), 1)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_x25_y25_a20_c75_h36
36
Grounding...


Current agent: 20 - at((1,9),0) => at((3,10),36): 100%|██████████| 20/20 [00:01<00:00, 17.96it/s]   


SAT
999 7 0.05
[((21, 7), 26), ((6, 15), 12), ((11, 5), 4), ((4, 10), 17), ((11, 4), 17), ((11, 5), 16), ((5, 10), 18)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 14.05it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True




## Approach 10: additional paths, simple threshold elimination, precomputed path +  subgraph with corridor and diamond

In [19]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content, additional_paths_computation=True)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        r,s = solver(r,'strategies/diamond.lp', additional_encodings=['strategies/corridor.lp', 'strategies/determination.lp'])
        
        rows.append({
            'Step Name' : 'strategy',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })

        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp',  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        print('\n')
        write_csv(f'benchmarks_result/{instance_name}-approach10.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54
Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:08<00:00,  4.20it/s]


SAT
2795 50 0.02857142857142857
[((36, 17), 53), ((36, 17), 52), ((36, 17), 51), ((36, 17), 50), ((36, 17), 49), ((36, 17), 48), ((36, 17), 47), ((36, 17), 46), ((36, 17), 45), ((36, 17), 44), ((36, 17), 43), ((36, 17), 42), ((36, 17), 41), ((36, 17), 40), ((36, 17), 39), ((22, 24), 39), ((36, 17), 38), ((36, 17), 37), ((36, 17), 36), ((36, 17), 35), ((36, 17), 34), ((36, 17), 33), ((36, 17), 32), ((36, 17), 31), ((36, 17), 30), ((36, 17), 29), ((28, 24), 5), ((34, 7), 10), ((15, 10), 42), ((15, 10), 40), ((23, 16), 30), ((24, 16), 29), ((36, 19), 27), ((36, 20), 26), ((36, 17), 25), ((36, 17), 23), ((37, 12), 19), ((37, 12), 17), ((35, 10), 2), ((34, 8), 11), ((34, 7), 12), ((37, 12), 37), ((37, 12), 35), ((26, 6), 43), ((26, 6), 41), ((32, 4), 2), ((28, 24), 4), ((28, 24), 3), ((33, 4), 3), ((34, 7), 11)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Groun

Current agent: 15 - at((18,7),0) => at((17,17),43): 100%|██████████| 15/15 [00:00<00:00, 28.24it/s] 


SAT
852 39 0.06666666666666667
[((16, 15), 20), ((19, 7), 41), ((2, 8), 37), ((18, 11), 36), ((2, 7), 36), ((2, 6), 35), ((2, 5), 34), ((2, 4), 33), ((2, 3), 32), ((2, 2), 31), ((3, 2), 30), ((4, 2), 29), ((14, 11), 27), ((14, 11), 26), ((18, 11), 23), ((19, 7), 18), ((17, 18), 14), ((19, 7), 10), ((17, 18), 42), ((19, 7), 24), ((16, 15), 22), ((16, 14), 21), ((18, 11), 29), ((11, 11), 29), ((11, 11), 1), ((19, 15), 2), ((11, 11), 2), ((14, 11), 32), ((19, 15), 4), ((19, 15), 3), ((18, 11), 30), ((12, 11), 29), ((18, 16), 6), ((14, 11), 31), ((17, 18), 41), ((14, 11), 14), ((16, 15), 35), ((19, 7), 25), ((19, 7), 8)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is False
Grounding...
Solving...
Done
Solver is False
Grounding...
Solving...
Done
Solver is False
Grounding...
Solving...
Done
Solver is False


random_x20_y20_a15_c85_h21
21
Grounding...


...

SAT
103 8 0.2
[((8, 10), 5), ((8, 10), 4), ((8, 10), 3), ((8, 10), 6), ((6, 3), 8), ((8, 10), 7), ((9, 10), 2), ((9, 9), 1)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_x25_y25_a20_c75_h36
36
Grounding...


Current agent: 20 - at((1,9),0) => at((3,10),36): 100%|██████████| 20/20 [00:01<00:00, 15.49it/s]   


SAT
1170 13 0.05
[((21, 7), 26), ((4, 10), 17), ((11, 5), 6), ((7, 15), 13), ((6, 15), 12), ((13, 21), 4), ((13, 22), 3), ((4, 14), 2), ((4, 14), 1), ((11, 5), 16), ((11, 5), 4), ((11, 4), 17), ((5, 10), 18)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.31it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True




## Approach 11: no additional paths, summed heatmap elimination, precomputed path

In [20]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  select_portion_based_on_heatmap(lgh['summed_heatmap_per_path'])
        
        rows.append({
            'Step Name' : 'select_portion_based_on_heatmap',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        # print(r)
        
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph=False, precomputed_path='mapf/precomputed_path_allgraph.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        write_csv(f'benchmarks_result/{instance_name}-approach11.csv', header,rows)

        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph=False, precomputed_path='mapf/precomputed_path_allgraph.lp',  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        

                write_csv(f'benchmarks_result/{instance_name}-approach11.csv', header,rows)

        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e, e.with_traceback)
        pool.terminate()

random_s40_a35_c75_h54
54


Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:04<00:00,  8.09it/s]


SAT
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



...

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 16.38it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 12: No additional path,summed heatmap elimination, subgraph approach

In [21]:


for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  select_portion_based_on_heatmap(lgh['summed_heatmap_per_path'])
        
        rows.append({
            'Step Name' : 'select_portion_based_on_heatmap',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        # r,s = solver(r,'mapf/add_lowest_heatmap_path.lp')
        
        print()
        # print(r)
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        write_csv(f'benchmarks_result/{instance_name}-approach12.csv', header,rows)
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        
                write_csv(f'benchmarks_result/{instance_name}-approach12.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54
Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:04<00:00,  8.04it/s]


SAT
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_s20_a15_c50_h43
43
Grounding...


...

Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.14it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 13: no additional paths, summed heatmap elimination, precomputed path on subgraph

In [22]:

for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  select_portion_based_on_heatmap(lgh['summed_heatmap_per_path'])
        
        rows.append({
            'Step Name' : 'select_portion_based_on_heatmap',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,           
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        print()
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp', horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        write_csv(f'benchmarks_result/{instance_name}-approach13.csv', header,rows)
        
    
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54


Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:04<00:00,  8.24it/s]


SAT
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



...

Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 13.93it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 14:  additional path, summed heatmap elimination, subgraph approach

In [23]:

for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content, additional_paths_computation=True)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  select_portion_based_on_heatmap(lgh['summed_heatmap_per_path'])
        
        rows.append({
            'Step Name' : 'select_portion_based_on_heatmap',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        

        write_csv(f'benchmarks_result/{instance_name}-approach14.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54
Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:08<00:00,  3.94it/s]


SAT
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_s20_a15_c50_h43
43
Grounding...


...

Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 14.99it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 15: no additional paths, summed heatmap elimination, precomputed path +  subgraph with corridor and diamond


In [24]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content, additional_paths_computation=True)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  select_portion_based_on_heatmap(lgh['summed_heatmap_per_path'])

        
        rows.append({
            'Step Name' : 'select_portion_based_on_heatmap',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        r,s = solver(r,'strategies/diamond.lp', additional_encodings=['strategies/corridor.lp', 'strategies/determination.lp'])
        
        rows.append({
            'Step Name' : 'strategy',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })

        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp',  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        
        write_csv(f'benchmarks_result/{instance_name}-approach15.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54


Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:08<00:00,  4.16it/s]


SAT
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



...

Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 14.30it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


In [25]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content, additional_paths_computation=True)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  select_portion_based_on_heatmap(lgh['summed_heatmap_per_path'])

        
        rows.append({
            'Step Name' : 'select_portion_based_on_heatmap',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        r,s = solver(r,'strategies/diamond.lp', additional_encodings=['strategies/corridor.lp', 'strategies/determination.lp'])
        
        rows.append({
            'Step Name' : 'strategy',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })

        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        
        write_csv(f'benchmarks_result/{instance_name}-approach16.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

random_s40_a35_c75_h54
54


Grounding...


Current agent: 35 - at((24,39),0) => at((24,18),54): 100%|██████████| 35/35 [00:07<00:00,  4.40it/s]


SAT
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



...

Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 14.04it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
